In [1]:
import plaidml.keras
plaidml.keras.install_backend()
# from tensorflow import keras
# keras.__version__

from keras.datasets import fashion_mnist
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D
from keras import backend as K

import matplotlib
matplotlib.use("Agg")
 
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.datasets import fashion_mnist
from keras.utils import np_utils
from keras import backend as K
# from keras.applications import ResNet50
import keras.applications as kapp
import matplotlib.pyplot as plt
import numpy as np
import cv2

import wandb

from data_loader import DataLoader

wandb: WARNING Keras version 2.2.4 is not fully supported. Required keras >= 2.4.0


In [2]:
def define_model(input_shape, numclasses):
    input_tensor = Input(shape=input_shape)
    # introduce a additional layer to get from bands to 3 input channels
    input_tensor = Conv2D(3, (1, 1))(input_tensor)

    base_model_resnet50 = kapp.ResNet50(include_top=False,
                              weights='imagenet',
                              input_shape=(100, 100, 3))
    base_model = kapp.ResNet50(include_top=False,
                     weights=None,
                     input_tensor=input_tensor)

    for i, layer in enumerate(base_model_resnet50.layers):
        # we must skip input layer, which has no weights
        if i == 0:
            continue
        base_model.layers[i+1].set_weights(layer.get_weights())

    # add a global spatial average pooling layer
    top_model = base_model.output
    top_model = GlobalAveragePooling2D()(top_model)

    # let's add a fully-connected layer
    top_model = Dense(2048, activation='relu')(top_model)
    top_model = Dense(2048, activation='relu')(top_model)
    # and a logistic layer
    predictions = Dense(numclasses, activation='sigmoid')(top_model)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    model.summary()
    return model

In [10]:
import wandb
from wandb.keras import WandbCallback 

wandb.init(project='mnist_test', sync_tensorboard=True)
config = wandb.config

# Variables definition
config.batch_size = 20
config.learning_rate = 0.001
config.input_shape = (28,28,1)
config.numclasses=10

wandb: wandb version 0.10.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


batch size 20, learning_rate 0.001, augment True


In [5]:
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
 
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
 
# one-hot encode the training and testing labels
trainY = np_utils.to_categorical(trainY, 10)
testY = np_utils.to_categorical(testY, 10)
 
# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "sneaker", "bag", "ankle boot"]


[INFO] loading Fashion MNIST...


In [7]:
trainX.shape

In [11]:
# initialize the optimizer and model
model = define_model(config.input_shape, config.numclasses)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 3)    6           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 34, 34, 3)    0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 14, 14, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (

In [12]:
model.compile(loss="categorical_crossentropy", optimizer='adam',metrics=["accuracy"])

In [13]:
history = model.fit(trainX, trainY,
                    epochs=2, 
#                     callbacks=callbacks_list,
                   ) 

Epoch 1/2


INFO:plaidml:Analyzing Ops: 1868 of 7655 operations complete
INFO:plaidml:Analyzing Ops: 3875 of 7655 operations complete


60000/60000 [==============================] - 254s 4ms/step - loss: 16.1108 - acc: 0.1000
Epoch 2/2
60000/60000 [==============================] - 286s 5ms/step - loss: 16.1181 - acc: 0.1000
